In [1]:
import pandas as pd
from scipy.spatial import cKDTree

data = pd.read_csv("../merged_datasets.csv")

tree = cKDTree(data[['longitude','latitude']].astype(float))

pairs = tree.query_pairs(.001, p=2)

In [2]:
def combineDisjointedSets(pairs):
    pairDict = {}
    for pair in pairs:
        if pair[0] not in pairDict:
            pairDict[pair[0]] = {pair[0],pair[1]}
            pairDict[pair[1]] = {pair[0]}
        else: 
            pointer = pair[0]
            while (len(pairDict[pointer]) == 1):

                pointer = next(iter(pairDict[pointer]))
            pairDict[pointer].add(pair[1])
            pairDict[pair[1]] = {pointer}
    return {k:x for k,x in pairDict.items() if len(x)>1}

In [3]:
newpairs = combineDisjointedSets(pairs)

In [4]:
data["group"] = ""
for k,x in newpairs.items():
    for xx in x:
        data["group"][xx] = k

<ipython-input-4-b3ac86ecc682>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["group"][xx] = k


In [5]:
from scipy.spatial.distance import pdist, squareform
from fuzzywuzzy import fuzz
import numpy as np
from itertools import combinations

def specialFuzz(x,y):
    return (fuzz.ratio(x,y) + fuzz.partial_token_sort_ratio(x,y))/2

newpairs2 = {}

groups = [x for x in data["group"].unique() if x != '']
for group in groups:
    names = np.array(data[data["group"]==group]["name"].values).reshape(-1,1)
    ids = np.array(data[data["group"]==group].index).reshape(-1,1)
    Y = pdist(names, specialFuzz)
    pairwise = list(combinations(range(len(names)),2))
    for i,y in enumerate(Y):
        if y>75:
            for ii in pairwise[i]:
                if group in newpairs2:
                    newpairs2[group].add(ids[ii][0])
                else:
                    newpairs2[group] = set(ids[ii])
                

c:\users\maxac\appdata\local\programs\python\python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
data["group2"] = ""
for k,x in newpairs2.items():
    for xx in x:
        data["group2"][xx] = k

<ipython-input-6-d1cb4d205c60>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["group2"][xx] = k


In [7]:
data.to_csv("FoodGroups2.csv")